In [51]:
import re
import csv
from functools import lru_cache

def real_sign(i, j):
    """Returns the sign for the priviously quaternion multiplication e_i * e_j. Now the real."""
    table = [
        [1]
    ]
    return table[i][j]

@lru_cache(maxsize=None)
def cayley_dickson_table(n):
    """Generates a multiplication table for the Cayley-Dickson algebra of dimension 2^n."""
    if n == 0:  # Base case: Quaternion table
        dim = 1
        return [[(real_sign(i, j), i ^ j) for j in range(dim)] for i in range(dim)]

    prev_table = cayley_dickson_table(n - 1)
    prev_dim = 1 << (n - 1)
    dim = 1 << n
    table = [[None] * dim for _ in range(dim)]

    for i in range(prev_dim):
        for j in range(prev_dim):
            base_val = prev_table[i][j]
            table[i][j] = base_val  # Block 'a'
            new_val = (-base_val[0], base_val[1] + prev_dim)
            table[i][j + prev_dim] = new_val  # Block 'b'
            table[i + prev_dim][j] = new_val  # Block 'c'
            table[i + prev_dim][j + prev_dim] = new_val  # Block 'd'

    for i in range(dim):
        local_row = i % prev_dim
        top = (i < prev_dim)
        for j in range(dim):
            local_col = j % prev_dim
            left = (j < prev_dim)
            s, _ = table[i][j]
            if i == 0 or j == 0:
                s = 1
            if i == j:
                s = 1 if i == 0 else -1
            in_a = top and left
            in_b = top and not left
            in_c = not top and left
            in_d = not top and not left
            if local_row == local_col:
                if in_a or in_b:
                    if n > 0 and local_row == 0:
                        s = real_sign(local_row, local_col)
                    elif local_row == 0:
                        s = 1
                    else:
                        s = -1
                elif in_c:
                    s = 1
                elif in_d:
                    s = -1
            if local_col == 0:
                if in_a or in_b:
                    s = 1
                elif in_c or in_d:
                    s = -1 if local_row == 0 else 1
            if local_row == 0:
                if in_a or in_b or (in_c and local_col == 0):
                    s = 1
                elif in_d:
                    s = -1
            if in_d:
                if local_row == 0 and local_col > 0:
                    s = 1
                if local_col == 0 and local_row > 0:
                    s = -1
            table[i][j] = (s, i ^ j)

    return table

def multiply_elements(a, b, n):
    """Multiplies two hypercomplex elements 'a' and 'b'."""
    table = cayley_dickson_table(n)
    dim = 1 << n
    result = [0] * dim
    for i in range(dim):
        ai = a[i]
        if ai == 0:
            continue
        for j in range(dim):
            bj = b[j]
            if bj == 0:
                continue
            coeff = ai * bj
            s, k = table[i][j]
            result[k] += coeff * s
    return result

def format_element(elem):
    """
    Returns a formatted string representation of a hypercomplex element given
    its list of coefficients.
    
    For example, [1, 2, -3, 0] becomes "1 + 2e1 - 3e2".
    """
    terms = []
    for i, coeff in enumerate(elem):
        if coeff == 0:
            continue
        if i == 0:
            term = f"{coeff}"
        else:
            if coeff == 1:
                term = f"e{i}"
            elif coeff == -1:
                term = f"-e{i}"
            else:
                term = f"{coeff}e{i}"
        terms.append(term)
    if not terms:
        return "0"
    formatted = terms[0]
    for term in terms[1:]:
        formatted += " - " + term[1:] if term.startswith("-") else " + " + term
    return formatted
def parse_element(s, dim):
    """Parses a hypercomplex element string into a list of coefficients."""
    s = s.replace(" ", "")
    if not s:
        raise ValueError("Empty input")
    if s[0] not in "+-":
        s = "+" + s
    tokens = re.findall(r'[+-][^+-]+', s)
    coeffs = [0.0] * dim
    for token in tokens:
        if 'e' in token:
            coeff_str, index_str = token.split('e', 1)
            #coeff = 1.0 if coeff_str in ("+", "-") else float(coeff_str)
            if(coeff_str in ("+")):
                coeff=1.0
            elif (coeff_str in ("-")):
                coeff=-1.0
            else:
                coeff=float(coeff_str)
            index = int(index_str)
        else:
            coeff = float(token)
            index = 0
        if index < 0 or index >= dim:
            raise ValueError(f"Index {index} out of range")
        coeffs[index] += coeff
    return coeffs

def save_table_to_csv(table, filename):
    """Saves the Cayley-Dickson multiplication table to a CSV file."""
    dim = len(table)
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write header row (e0, e1, e2, ...)
        header = [f"e{j}" for j in range(dim)]
        writer.writerow([""] + header) # Empty cell in top-left corner

        # Write table data
        for i in range(dim):
            row = [f"e{i}"]  # First element in the row is the basis element
            for sign, index in table[i]:
                row.append(f"{'+' if sign == 1 else '-' }e{index}")
            writer.writerow(row)

def main():
    print("Hypercomplex Element Multiplication & Cayley-Dickson Table Generator")

    try:
        n = int(input("Enter n (n=1 for Complex, n=2 for Quaternions, n=3 for Octonions, etc.): "))
    except ValueError:
        print("Error: n must be an integer.")
        return

    dim = 1 << n
    print(f"\nThis algebra has dimension {dim} (basis elements e0, e1, ..., e{dim-1}).\n")

    # Generate and save the table
    table = cayley_dickson_table(n)
    filename = f"cayley_dickson_table_n{n}.csv"
    save_table_to_csv(table, filename)
    print(f"Cayley-Dickson multiplication table saved to '{filename}'")
    print(f"\nCayley–Dickson Multiplication Table (n={n}, dimension={dim}):")
    for row in table:
        print("\t".join(f"{'' if s == 1 else '-'}e{k}" for s, k in row))
    print("-" * 40)
    # Optional: Perform multiplication (kept from original script)
    while True:
        try:
            a_str = input("Enter the first element (e.g., '1 + 2e1 - 3e2', or 'q' to quit):\n")
            if a_str.lower() == 'q':
                break
            a = parse_element(a_str, dim)

            b_str = input("\nEnter the second element (e.g., '4 - e1 + 0.5e3'):\n")
            b = parse_element(b_str, dim)

            prod = multiply_elements(a, b, n)
            print("\nResult of multiplication:")
            print(f"({format_element(a)}) * ({format_element(b)}) = {format_element(prod)}")

        except ValueError as e:
            print("Error:", e)

if __name__ == "__main__":
    main()

Hypercomplex Element Multiplication & Cayley-Dickson Table Generator

This algebra has dimension 16 (basis elements e0, e1, ..., e15).

Cayley-Dickson multiplication table saved to 'cayley_dickson_table_n4.csv'

Cayley–Dickson Multiplication Table (n=4, dimension=16):
e0	e1	e2	e3	e4	e5	e6	e7	e8	e9	e10	e11	e12	e13	e14	e15
e1	-e0	e3	-e2	e5	-e4	-e7	e6	e9	-e8	-e11	e10	-e13	e12	e15	-e14
e2	-e3	-e0	e1	e6	e7	-e4	-e5	e10	e11	-e8	-e9	-e14	-e15	e12	e13
e3	e2	-e1	-e0	e7	-e6	e5	-e4	e11	-e10	e9	-e8	-e15	e14	-e13	e12
e4	-e5	-e6	-e7	-e0	e1	e2	e3	e12	e13	e14	e15	-e8	-e9	-e10	-e11
e5	e4	-e7	e6	-e1	-e0	-e3	e2	e13	-e12	e15	-e14	e9	-e8	e11	-e10
e6	e7	e4	-e5	-e2	e3	-e0	-e1	e14	-e15	-e12	e13	e10	-e11	-e8	e9
e7	-e6	e5	e4	-e3	-e2	e1	-e0	e15	e14	-e13	-e12	e11	e10	-e9	-e8
e8	-e9	-e10	-e11	-e12	-e13	-e14	-e15	-e0	e1	e2	e3	e4	e5	e6	e7
e9	e8	-e11	e10	-e13	e12	e15	-e14	-e1	-e0	-e3	e2	-e5	e4	e7	-e6
e10	e11	e8	-e9	-e14	-e15	e12	e13	-e2	e3	-e0	-e1	-e6	-e7	e4	e5
e11	-e10	e9	e8	-e15	e14	-e13	e12	-e3	-e2	e1	-e0	-e7	e6	-e

In [56]:
import re
import csv
from functools import lru_cache

# (cayley_dickson_table, get_element, format_element, parse_element, 
#  quaternion_sign, are_equal_elements, are_opposite_elements functions - same as before)
#  (omitting for brevity, but include in your script)

def quaternion_sign(i, j):
    """Returns the sign for the quaternion multiplication e_i * e_j."""
    table = [
        [1, 1, 1, 1],
        [1, -1, 1, -1],
        [1, -1, -1, 1],
        [1, 1, -1, -1]
    ]
    return table[i][j]

@lru_cache(maxsize=None)
def cayley_dickson_table(n):
    """Generates a multiplication table for the Cayley-Dickson algebra of dimension 2^n."""
    if n == 2:  # Base case: Quaternion table
        dim = 4
        return [[(quaternion_sign(i, j), i ^ j) for j in range(dim)] for i in range(dim)]

    prev_table = cayley_dickson_table(n - 1)
    prev_dim = 1 << (n - 1)
    dim = 1 << n
    table = [[None] * dim for _ in range(dim)]

    for i in range(prev_dim):
        for j in range(prev_dim):
            base_val = prev_table[i][j]
            table[i][j] = base_val
            new_val = (-base_val[0], base_val[1] + prev_dim)
            table[i][j + prev_dim] = new_val
            table[i + prev_dim][j] = new_val
            table[i + prev_dim][j + prev_dim] = new_val

    for i in range(dim):
        local_row = i % prev_dim
        top = (i < prev_dim)
        for j in range(dim):
            local_col = j % prev_dim
            left = (j < prev_dim)
            s, _ = table[i][j]
            if i == 0 or j == 0:
                s = 1
            if i == j:
                s = 1 if i == 0 else -1
            in_a = top and left
            in_b = top and not left
            in_c = not top and left
            in_d = not top and not left
            if local_row == local_col:
                if in_a or in_b:
                    if n > 2 and local_row < 4:
                        s = quaternion_sign(local_row, local_col)
                    elif local_row == 0:
                        s = 1
                    else:
                        s = -1
                elif in_c:
                    s = 1
                elif in_d:
                    s = -1
            if local_col == 0:
                if in_a or in_b:
                    s = 1
                elif in_c or in_d:
                    s = -1 if local_row == 0 else 1
            if local_row == 0:
                if in_a or in_b or (in_c and local_col == 0):
                    s = 1
                elif in_d:
                    s = -1
            if in_d:
                if local_row == 0 and local_col > 0:
                    s = 1
                if local_col == 0 and local_row > 0:
                    s = -1
            table[i][j] = (s, i ^ j)

    return table

def get_element(n, i, j):
    """Retrieves the element at row i and column j of the Cayley-Dickson multiplication table."""
    table = cayley_dickson_table(n)
    sign, index = table[i][j]
    return sign, index

def format_element(elem):
    """Formats a hypercomplex element into a human-readable string."""
    terms = []
    for i, coeff in enumerate(elem):
        if coeff == 0:
            continue
        if i == 0:
            term = f"{coeff}"
        else:
            term = f"{'+' if coeff > 0 else ''}{coeff}e{i}" if coeff != 1 and coeff != -1 else f"{'+' if coeff>0 else ''}e{i}"
            term = term.replace("+-","-")
            term = term.replace("++","+")
        terms.append(term)

    if not terms:
        return "0"
    return "".join(terms)

def parse_element(s, dim):
    """Parses a hypercomplex element string into a list of coefficients."""
    s = s.replace(" ", "")
    if not s:
        raise ValueError("Empty input")
    if s[0] not in "+-":
        s = "+" + s
    tokens = re.findall(r'[+-][^+-]+', s)
    coeffs = [0.0] * dim
    for token in tokens:
        if 'e' in token:
            coeff_str, index_str = token.split('e', 1)
            #coeff = 1.0 if coeff_str in ("+", "-") else float(coeff_str)
            if(coeff_str in ("+")):
                coeff=1.0
            elif (coeff_str in ("-")):
                coeff=-1.0
            else:
                coeff=float(coeff_str)
            index = int(index_str)
        else:
            coeff = float(token)
            index = 0
        if index < 0 or index >= dim:
            raise ValueError(f"Index {index} out of range")
        coeffs[index] += coeff
    return coeffs

def are_equal_elements(elem1, elem2):
    """Checks if two elements (sign, index tuples) are considered "equal"."""
    sign1, index1 = elem1
    sign2, index2 = elem2
    return index1 == index2 and sign1 == sign2

def are_opposite_elements(elem1, elem2):
    """Checks if two elements are considered "opposite" (same index, opposite signs)."""
    sign1, index1 = elem1
    sign2, index2 = elem2
    return index1 == index2 and sign1 != sign2

def multiply_elements(a, b, n):
    """Multiplies two hypercomplex elements 'a' and 'b'."""
    table = cayley_dickson_table(n)
    dim = 1 << n
    result = [0] * dim
    for i in range(dim):
        ai = a[i]
        if ai == 0:
            continue
        for j in range(dim):
            bj = b[j]
            if bj == 0:
                continue
            coeff = ai * bj
            s, k = table[i][j]
            result[k] += coeff * s
    return result
def find_zero_divisor_expressions_exploratory_v4(n_dim, start_i, start_j, search_offset_limit):
    """
    Explores zero divisor expressions by checking 'equal'/'opposite' elements 
    in a search region around e_(start_i, start_j) with varying offsets.
    """
    zero_divisor_expressions = []
    dim = 1 << n_dim

    q = get_element(n_dim, start_i, start_j) # Get the reference element q

    for n_offset_power in range(3, search_offset_limit + 1): # Iterate through n values (block scales/offsets)
        n_offset = 1 << n_offset_power

        search_rows = range(start_i, min(start_i + search_offset_limit * 8, dim)) # Define search range for rows
        search_cols = range(start_j, min(start_j + search_offset_limit * 8, dim)) # Define search range for cols


        for found_i in search_rows:
            for found_j in search_cols:
                if found_i == start_i and found_j == start_j: # Skip the starting element itself
                    continue

                t = get_element(n_dim, found_i, found_j) # Get the element t at (found_i, found_j)


                if q[1] == t[1]: # Compare basis element indices
                    if are_equal_elements(q, t): # Case 1: q and t are "equal"
                        expression = f"(e{start_i}{format_element(parse_element('+1e'+str(found_i),dim))}) * (e{start_j}-{format_element(parse_element('-1e'+str(found_j),dim))}) = 0"
                        a=f"e{start_i}{format_element(parse_element('+1e'+str(found_i),dim))}"
                        b=f"e{start_j}-{format_element(parse_element('-1e'+str(found_j),dim))}"
                        #print(a,b)
                        a=parse_element(a,2**n_dim)
                        b=parse_element(b,2**n_dim)
                        prod = multiply_elements(a, b, n_dim)
                        #print(format_element(prod))
                        if(format_element(prod)=="0"):
                            zero_divisor_expressions.append(expression)
                    elif are_opposite_elements(q, t): # Case 2: q and t are "opposite"
                        expression = f"(e{start_i}{format_element(parse_element('+1e'+str(found_i),dim))}) * (e{start_j}{format_element(parse_element('+1e'+str(found_j),dim))}) = 0"
                        a=f"e{start_i}{format_element(parse_element('+1e'+str(found_i),dim))}"
                        b=f"e{start_j}{format_element(parse_element('+1e'+str(found_j),dim))}"
                        #print(a,b)
                        a=parse_element(a,2**n_dim)
                        b=parse_element(b,2**n_dim)
                        prod = multiply_elements(a, b, n_dim)
                        #print(format_element(prod))
                        if format_element(prod)=="0":
                            zero_divisor_expressions.append(expression)

    return zero_divisor_expressions



def main():
    print("Cayley-Dickson Exploratory Zero Divisor Expression Finder (V4 - Exploratory Scan)")
    print("When selecting element based on i, j it doesn't goes backword it goes to i+n, j+n")
    print("offeset is by how many 8x8 you want to search max=2^n/8 e.g n=6, max=2^6/8=8")
    print("So it better for full zero divisors is to select elements from upper triangle n/2 x n/2 and set to maximum offset")
    try:
        n_dim = int(input("Enter the dimension level n (n > 3): "))
        if n_dim <= 3:
            print("Dimension level n must be greater than 3.")
            return
        start_i = int(input(f"Enter starting row index i: max={(2**n_dim)-1} "))
        start_j = int(input(f"Enter starting column index j: max={2**n_dim-1}"))
        search_offset_limit = int(input(f"Enter search offset limit (power of 2, e.g., 4 for up to 16x16 search):max={((2**n_dim)//4)} "))
        dim = 1 << n_dim
        if start_i < 0 or start_i >= dim or start_j < 0 or start_j >= dim:
            print(f"Indices i and j are out of range for dimension {dim}")
            return
        if search_offset_limit < 3:
            print(f"Search offset limit (power of 2) must be at least 3 (8x8 blocks) offeset is 2^n means the depth of 8x8 it goes.max={(2**n_dim)//8}")
            return


    except ValueError as e:
        print("Invalid input:", e)
        return

    expressions = find_zero_divisor_expressions_exploratory_v4(n_dim, start_i, start_j, search_offset_limit)

    if expressions:
        
        print(f"\nExploratory Zero divisor expressions related to cell e_{start_i},e_{start_j} in A_{n_dim}:")
        expressions = list(dict.fromkeys(expressions))
        print("zero pair divsors = ",len(expressions),"divsors")
        
        for expr in expressions:
            
            print(expr)
    else:
        print(f"\nNo exploratory zero divisor expressions found for cell e_{start_i},e_{start_j} in A_{n_dim} based on the current method and search range.")


if __name__ == "__main__":
    main()

Cayley-Dickson Exploratory Zero Divisor Expression Finder (V4 - Exploratory Scan)
When selecting element based on i, j it doesn't goes backword it goes to i+n, j+n
offeset is by how many 8x8 you want to search max=2^n/8 e.g n=6, max=2^6/8=8
So it better for full zero divisors is to select elements from upper triangle n/2 x n/2 and set to maximum offset

Exploratory Zero divisor expressions related to cell e_6,e_7 in A_4:
zero pair divsors =  4 divsors
(e6+e10) * (e7-e11) = 0
(e6+e11) * (e7+e10) = 0
(e6+e12) * (e7-e13) = 0
(e6+e13) * (e7+e12) = 0
